# Rank-normalization, folding, and localization: An improved $\widehat{R}$ for assessing convergence of MCMC by Vehtari et al. 2019

### Students: Tanveer Karim and Ian Weaver
### Course: APMTH 207
### GitHub Repository: https://github.com/icweaver/pyhat
### Paper: https://ui.adsabs.harvard.edu/abs/2019arXiv190308008V/abstract

## Problem Statement and Existing Work
The traditional $\widehat R$ statistic was first defined by [Gelman and Rubin (1992)](https://www.jstor.org/stable/2246093?seq=1#metadata_info_tab_contents) and the subsequent revision involving the split variation was defined by [Gelman et al. (2013)](https://books.google.com/books/about/Bayesian_Data_Analysis.html?id=eSHSBQAAQBAJ). These statistics theoretically measure the variance between chains and in chains for an MCMC model and quantifies whether the chains have not mixed well, i.e. whether they have diverged. Along with traceplots, these statistics serve as a powerful way to diagnose whether the MCMC samples can be used for subsequent analysis. 

However, [Vehtari et al. (2019)](https://ui.adsabs.harvard.edu/abs/2019arXiv190308008V/abstract) showed that the well-known $\widehat R$ statistic and traceplots have limitations and fail in many cases. Thus, they propose an alternative definition of $\widehat R$ and introduce rankplots as an alternative to traceplots as visual diagnostics. 

## Context and Unique Contribution

For anyone doing Bayesian analysis over multidimensions, MCMC samplers are paramount for modelling purposes. Beyond 2D, visualizing convergence of MCMC samplers on the parameter space becomes very complicated and we have to resort to alternate methods to understand convergence. Hence, if our well-used statistic and diagnostics are not performing as we expect them to, then almost any complex multidimensional MCMC modelling becomes suspect. Because of this importance, we decided to tackle this paper and understand how to best study convergence issues related to MCMC samplers. 

## Technical Content
The key problems that [Vehtari et al. (2019)](https://ui.adsabs.harvard.edu/abs/2019arXiv190308008V/abstract) identified with the existing $\widehat{R}$ is that it was only sensitive to errors in the first moment and it failed catastrophically when the variance was infinite. Their proposed $\widehat{R}$ utilizes the concept of rank normalization. 

Rank normalization essentially converts the chains from values to ranks for a given parameter and then converts the ranks to quantiles of a standard normal. While this procedure destroys any information about the values within the chains themselves, it nevertheless converts the chains to nice standard normals which are easy to operate with. Afterwards, we can run the traditional $\widehat{R}$ formula on the rank normalized values to get the ranked $\widehat{R}$. 

This conversion makes the rank normalized $\widehat{R}$ robust against heavy tails and makes it sensitive to changes in tails, especially for the folded ranked $\widehat{R}$ statistic, which is sensitive to scale. Moreover, rank normalization makes $\widehat{R}$ parameterization invariant. 

In addition, the rankplots are generated by producing histograms of ranks of $n$ chains. If the chains sample the same underlying distribution, then the rankplots will look similar because they would randomly sample the space the same way after burn-in and thinning. Any deviation from this behaviour would imply that different chains are exploring different parts of the parameter space and have not mixed in well. 

The high-level organization of our codebase [`pyhat`](https://github.com/icweaver/pyhat) is shown below and described in detail in the associated notebooks:
```
pyhat
├── Project_Summary_Notebook.ipynb
├── README.md
├── codes
│   ├── plotutils.py
│   └── utils.py
└── examples
    ├── multiplanet
    │   ├── data
    │   │   ├── map_solution.npy
    │   │   ├── multiplanet_chain_1.pkl
    │   │   ├── multiplanet_chain_2.pkl
    │   │   ├── multiplanet_chain_3.pkl
    │   │   ├── multiplanet_chain_4.pkl
    │   │   └── trace.pkl
    │   └── multiplanet.ipynb
    ├── rhat_variance
    │   ├── data
    │   │   └── models.npy
    │   └── rhat_variance.ipynb
    └── toymodel_gaussian
        ├── toymodel_2DGaussian.ipynb
        ├── toymodel_gaussian.ipynb
        └── utils.py
```

The main implementation and experiment notebooks detailing its use are:
* `codes/` - implementation of modified $\widehat R$ statistics proposed by paper and tools to visualize them
* [`examples/multiplanet/multiplanet.ipynb`](examples/multiplanet/multiplanet.ipynb) - domain specific application (astronomy) of the tools described above
* [`examples/rhat_variance/rhat_variance.ipynb`](examples/rhat_variance/rhat_variance.ipynb) - increased variance example
* [`examples/toymodel_gaussian/toymodel_2DGaussian.ipynb`](examples/toymodel_gaussian/toymodel_2DGaussian.ipynb) - presentation of $\widehat R$ and visualization tools on a 2D Gaussian with changing correlation
* [`examples/toymodel_gaussian/toymodel_gaussian.ipynb`](examples/toymodel_gaussian/toymodel_gaussian.ipynb) - presentation of $\widehat R$ and visualization tools on a simple Gaussian distribution with changing variance

Note: the `data` folders holds intermediate results that can be loaded into each notebook to avoid running time-intensive cells again. `examples/multiplanet/data/trace.pkl` was too large to upload to our Github repo, but we are more than happy to share it upon request.

**We recommend starting with [`examples/toymodel_gaussian/toymodel_gaussian.ipynb`](examples/toymodel_gaussian/toymodel_gaussian.ipynb) because it introduces and details the implementation of the modified $\widehat R$ diagnostic and visualization tools proposed by [Vehtari et al. (2019)](https://ui.adsabs.harvard.edu/abs/2019arXiv190308008V/abstract).**

## Experiments

In [`examples/toymodel_gaussian/toymodel_gaussian.ipynb`](examples/toymodel_gaussian/toymodel_gaussian.ipynb), we show a proof-of-concept of the new $\widehat{R}$ statistic along with rankplot to see whether [Vehtari et al. (2019)](https://ui.adsabs.harvard.edu/abs/2019arXiv190308008V/abstract)'s proposals make sense. We sampled a standard normal with $4$ chains and replaced $1$ of the chains with a chain of a different model that was sampled from a different normal distribution (different variance). Our hypothesis was that since both the regular chains and the fake chain share the same mean, the traditional $\widehat{R}$ would not pick up the difference in variance but the ranked $\widehat{R}$ would. It turned out that indeed our speculation was correct as the folded rank $\widehat{R}$ statistic is robust against changes in tails and picked up this fake chain easily. Moreover, the rankplots clearly showed which chain was having issues. 

In [`examples/widehat R_variance/rhat_variance.ipynb`](examples/rhat_variance/rhat_variance.ipynb), we repeated the above experiment for chain variances ranging from 1 to $\sqrt 2$ to probe where the $\widehat R$ and rankplot implementation began to misbehave. We found that for the toy model defined above, a chain with variance $\approx 1.4$ vs. the original $\sigma^2 = 1.0$ is where the divergence in the sampler begins to become apparent using the $\widehat{R}$ metric. In contrast, the rankplot picks up the difference earlier (around $\sigma \approx 1.3$) than the $\widehat{R}$ metric, suggesting that rankplots can indeed be a powerful complement to the $\widehat{R}$ metric.

While the previous experiment was sound and confirmed the paper, it was only in $1\text D$ and we wanted to scale it to see whether it worked for a $2\text D$ Gaussian. Furthermore, we wanted to explore the question as to whether these diagnostics would be able to understand the difference between two Gaussians with the same mean but different correlation between the features -- in our case one was isotropic and the other case was heavily correlated. We showcase this experiment in [`examples/toymodel_gaussian/toymodel_2DGaussian.ipynb`](examples/toymodel_gaussian/toymodel_2DGaussian.ipynb).

In this second experiment, we found that the rank $\widehat{R}$ and rankplots are not robust at picking up differences between isotropy and correlation. We injected a fake correlated chain into a model of an isotropic Gaussian. The metrics proposed in the paper were not able to pick up the difference. On the other hand, when we injected a fake correlated chain that also had a difference in variance, i.e. different diagonal terms compared to the isotropic Gaussian, then these diagnostic methods picked up the difference easily. This suggested that rank $\widehat{R}$ and rankplots are sensitive to diagonal terms in covariance matrices, but are not sensitive at all to off-diagonal terms.

In the final experiment, [`examples/multiplanet/multiplanet.ipynb`],(examples/multiplanet/multiplanet.ipynb) we test our implementations of $\widehat R$ on a real problem in astronomy: fitting for the [light curve](https://en.wikipedia.org/wiki/Light_curve) of a [multi-exoplanetary](https://en.wikipedia.org/wiki/Exoplanet) system. We used the open source astronomy package [`exoplanet`](https://en.wikipedia.org/wiki/Exoplanet) to accomplish this task for a toy two-planet exoplanet model with 11 dimensions. We used the `pymc3` HMC sampler combined with [Theano operations](http://deeplearning.net/software/theano/extending/extending_theano.html) wrapped in `exoplanet` to perform transit inference. The code also leveraged custom `pymc3` types defined in `exoplanet` to define the model. Our hypothesis was that the different variants of $\widehat R$ should all return similar values for each parameter because of the expected robustness of HMC for this well defined toy model. The result was as expected, as confirmed by the rank plots. The modified $\widehat R$ implementations generally tended to produce marginally larger values than the standard definition's as well.

## Evaluation

When we compare the results between the 1D and the 2D Gaussian cases, we see that the ranked $\widehat{R}$ and rankplots are sensitive to changes in variance terms but not in covariance terms. This makes sense because the way $\widehat{R}$ is defined, it uses marginalized information for a specific parameter. So for evaluating whether marginalized chains have mixed well or not, ranked $\widehat{R}$ and rankplots are quite powerful and prove the point of the paper. In addition, we saw that rankplots can detect divergence issues at a much lower sensitivity compared to $\widehat{R}$ statistic. This suggests that rankplot definitely has an upper hand in comparison to traceplots.

However, we found that one should exercise caution when using rank $\widehat{R}$ and rankplots at claiming that they indicate chains are sampling from the same underlying distribution. We clearly showed that even when chains explore different underlying distributions, the diagnostics could give values that are consistent with chains exploring the same distribution. **This implies that ranked $\widehat{R}$ and rankplots can only tell us whether chains have NOT mixed well**; it does not tell us anything about whether chains HAVE mixed well.

## Future Work

For future work, we would like to understand how to modify $\widehat{R}$ in a way such that it captures correlation terms from the covariance matrix. Right now, off-diagonal terms do not contribute at all, as shown in [toymodel_2DGaussian](examples/toymodel_gaussian/toymodel_2DGaussian.ipynb). We would like to explore and understand if there is a better way quantify changes in the correlation terms. One possible avenue is to understand the relationship between $\widehat{R}$ and autocorrelation function and see if there is a way to unify them together. 

In addition, we would like to explore what makes rankplots more sensitive to subtle changes in comparison to $\widehat{R}$. This understanding could help us better define $\widehat{R}$.